In [1]:
import sys
sys.path.append('../')
import momepy
from optimizer import *

/Users/polina/Documents/Uni/nir/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# загрузка интермодального графа ВО (drive+PT+walk)
import pickle
with open ('/Users/polina/Documents/Uni/nir/inter_spb_drive.pickle','rb') as f:
    res = pickle.load(f)
    
# загрузка кварталов ВО с населением и кол-вом сервисов
blocks = gpd.read_parquet('krupenya_blocks.parquet')

In [3]:
# Извлечение drive+BUS+walk
G = get_walk_bus_car_graph(res)

In [4]:
# Разделение на отдельные графы
walk_g, car_g, bus_g = separate_graphs(G)

In [5]:
# Выделение автобусных маршрутов
bus_n, bus_e = momepy.nx_to_gdf(bus_g) 
routes_list = extract_routes(bus_e)

In [6]:
# Проецирование автобусных остановок на drive граф  и выделение подграфа car-bus
# (чтобы найти пути между оставновками, не соединенными автобусными маршрутами)
G_proj = G.copy()
for i in bus_n.index:
    a = bus_n.iloc[[i]]
    G_proj = add_node_to_graph(G_proj, a, car_g)
carbus = create_subgraph(G_proj, ['car','bus'])

In [7]:
# Кодирование последовательности остановок в хромосому
long,splits = encode_routes(routes_list)

In [8]:
# все возможные модифицаии для ребер автобусного графа
gene_space_dict = possible_modifications(carbus, routes_list, bus_g)
gene_space = [gene_space_dict[i] for i in long] # gene space для генетического алгоритма  - как может поменяться каждый ген
path_to_id, id_to_path, gene_space_encoded = int_gene_space(gene_space) # кодирование генов из кортежей в числа

In [ ]:
# запуск метода
graf = genetic(carbus, splits, walk_g, blocks, id_to_path, gene_space_encoded, num_generations=10)

In [ ]:
# Визуализация результата
e = momepy.nx_to_gdf(graf[0])[1]
bus = e[e['type']=='bus']
bus[bus['desc']=='route 33'].explore()

In [ ]:
bus.explore('desc')

In [ ]:
1/0

ZeroDivisionError: division by zero

In [ ]:
e = momepy.nx_to_gdf(graf[0])[1]
bus = e[e['type']=='bus']
bus[bus['desc']=='route 33'].explore()

In [ ]:
bus.explore('desc')

In [ ]:
bus.explore('desc')